In [1]:
class Module:
    #fonction pour passer au 'module' suivant
    def forward(self,x):
        pass
    
    def backward_updateGradient(self,x,delta):
        pass
    
    def zeroGradParameters(self):
        pass
    
    #deriver de la fonction de forward 
    def backward_delta(self,x,delta):
        pass

    
class Loss :
    def getLossValue (self, ypredit ,y):
        pass
    def backward(self,ypredit,y):
        pass
    
class SquareLoss(Loss) :
    def __init__(self,dimension):
        self.dimension=dimension
        
    def getLossValue (self, y_predit,y):
        loss = 0.0
        for k in range(0,self.dimension) :
            loss += (y_predit[k]-y[k])*(y_predit[k]-y[k])
        loss /= self.dimension
        return loss
    
    # renvoi la valeur de delta
    # derive du loss
    def backward(self,y_predit, y):
        grad = np.zeros(self.dimension)
        for k in range(0,self.dimension):
            grad[k] = (2*(y_predit[k]-y[k]))/self.dimension
        return grad

    
# Module lineaire    
class LinearModule(Module):
    def __init__(self,n,C):
        self.X = n #input_dimension
        self.Y = C #ouput_dimension
        self.theta = np.zeros((self.X,self.Y))
        self.grad = np.zeros((self.X,self.Y))
        
    def zeroGradParameters(self):
        self.grad = np.zeros((self.X,self.Y))
    
    # produit matriciel
    def forward(self,x):
        result = np.dot(x,self.theta)
        return result
    
    # init theta avec des varleurs aleatoire
    def randomize(self, variance):
        for i in range(0,self.X):
            for j in range(0,self.Y):
                self.theta[i][j]= random.uniform(-variance,variance)
    
    # deriver de la fonction de forward 
    def backward_updateGradient(self,x,delta):
        #print(self.X)
        #print(self.Y)
        for i in range(0,self.X):
            #print(i)
            #print("x["+ str(i)+']')
            for j in range(0,self.Y):
                #print("delta["+ str(j)+']')
                self.grad[i][j] += delta[j]*x[i]
        return self.grad
    
    def updateParameters(self,learning_rate):
        self.theta = self.theta - learning_rate*self.grad
        
        
#Module tanh      
class TanHModule(Module):
    def __init__(self,dimension):
        self.X = dimension
     
    #tanh
    def forward(self,x):
        result = np.tanh(x)
        return result
    
    #(derive de than) * delta
    def backward_delta(self,x,delta):
        res = [0]*self.dimension
        print(x)
        for i in range(0,self.X):
            print(i)
            #deriver de than = 1- tan(x)*tan(x)
            res[i] = delta[i]*(1-tanh(x[i])*tanh(x[i]))
        return res 
    
   # init theta avec des varleurs aleatoire
    def randomize(self, variance):
        pass
    
    
    
import numpy as np
import random
import matplotlib.pyplot as plt
from math import tanh

#voir tme3 ou 4 de mapsi/arf ce sont des images plt.imgshow(vecteur.reshape)
def load_mnist(filename):
    with open(filename,"r") as f:
        f.readline()
        data =[ [float(x) for x in l.split()] for l in f if len(l.split())>2]
    tmp = np.array(data)
    return tmp[:,1:],tmp[:,0].astype(int)

#7000 matrices de 16*16
train = load_mnist('USPS/USPS_train.txt')
test = load_mnist('USPS/USPS_test.txt')

#donnees de train
x_train = train[0]
y_train = train[1]

#donnees de test
x_test = test[0]
x_test = test[1]
#print(x_train[0])
#print(y_train[1])



#affichge l'image
def display_image ( X ):
    """
    Etant donné un tableau X de 256 flotants représentant une image de 16x16
    pixels, la fonction affiche cette image dans une fenêtre.
    """
    # on teste que le tableau contient bien 256 valeurs
    if X.size != 256:
        raise ValueError ( "Les images doivent être de 16x16 pixels" )

    # on crée une image pour imshow: chaque pixel est un tableau à 3 valeurs
    # (1 pour chaque canal R,G,B). Ces valeurs sont entre 0 et 1
    Y = X / X.max ()
    img = np.zeros ( ( Y.size, 3 ) )
    for i in range ( 3 ):
        img[:,i] = X

    # on indique que toutes les images sont de 16x16 pixels
    img.shape = (16,16,3)

    # affichage de l'image
    plt.imshow( img )
    plt.show ()
    
m_tan = TanHModule(2)
m_tan.randomize(0.1)
x = [0.5,0.1]
delta = [1.,7.]
#print(m_tan.forward(x))
#print(m_tan.backward_delta(x,delta))

In [9]:

m_linear1 = LinearModule(256,500)
m_tan1 = TanHModule(500)
m_linear2 = LinearModule(500,10)
m_tan2 = TanHModule(10)
loss = SquareLoss(10)



gradient_step = 0.01
m_linear1.randomize(1) 
m_linear2.randomize(1) 

maxEpoch = 1 

for i in range(maxEpoch):
    j=random.randint(0,len(x_train)-1)
    x = x_train[j]
    y = y_train[j]
    Y = np.zeros(10)
    Y[y] = 1
    m_linear1.zeroGradParameters()   
    m_tan1.zeroGradParameters()
    m_linear2.zeroGradParameters()
    m_tan2.zeroGradParameters()
    
    prediction_linear1 = m_linear1.forward(x)
    #print(len(prediction_linear1))
    #print(prediction_linear1)
    
    prediction_tanh1 = m_tan.forward(prediction_linear1)
    #print(len(prediction_tan1))
    
    prediction_linear2 = m_linear2.forward(prediction_tanh1)
    #print(len(predicti))
    #print(prediction_linear2)
    
    prediction_tanh2 = m_tan.forward(prediction_linear2)
    #print(len(prediction_tan2))
    #print(prediction_tan2)
    
    error = loss.getLossValue(prediction_tanh2,Y)
    #print(error)
    
    delta_tanh2 = loss.backward(prediction_tanh2,Y)
    #print(delta_tanh2)
    
    print(prediction_linear2)
    print(delta_tanh2)
    delta_linear2 = m_linear2.backward_updateGradient(prediction_linear2, delta_tanh2)
    print(delta_linear2)
    
    #print(len(prediction_linear2))
    #print(delta_linear2[0])
    #print(m_linear2.theta[0])
    #print(len(m_linear2.theta[0]))
    #m_linear2.backward_updateGradient(prediction_tanh1 ,delta_linear2)
    
    #delta_tanh1 = loss.backward(prediction_tanh1,delta_tanh2)
    #print(delta_tanh1)
    
    #delta_linear1 = loss.backward(prediction_linear1, delta_tanh1)
    #print(delta_linear1)
    #print(len(x))
    #print(len(delta_linear1))
    #print(m_linear1.Y)
    #m_linear2.backward_updateGradient(x ,delta_tanh1)
    #print(delta_linear1)
    
    
    
    #prediction_tan2.backward_updateGradient(x,delta)
    #module.updateParameters(gradient_step)
            

[ 18.42840023   4.34412675   7.23558784  -3.04631968 -22.80198933
  16.5992331    4.73494176  -5.20425322 -16.5106171    3.89512511]
[  2.00000000e-01   1.99932590e-01   1.99999792e-01  -1.99098262e-01
  -2.00000000e-01  -1.53210777e-15   1.99969146e-01  -1.99987930e-01
  -2.00000000e-01   1.99834569e-01]


IndexError: index 10 is out of bounds for axis 0 with size 10

In [11]:
np.tanh([0.52,0.45])

array([ 0.47770001,  0.42189901])